In [ ]:
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ump2 = pd.read_csv("/data/zkp/ump2/detailhost/service.csv")

In [ ]:
machine = pd.read_csv("/data/zkp/ump2/detailhost/machine.csv")
print(machine.isnull().sum())

In [ ]:
ump2

In [ ]:
pod_meta = pd.read_csv("./pod_meta.csv")
ay49c = pod_meta[pod_meta.container_app_name=="ay49c"]

In [ ]:
test_nc_sn = {}
for ip in ump2.container_ip.unique().tolist():
    for i in range(pod_meta.shape[0]):
        if pod_meta.container_ip.values[i] == ip:
            test_nc_sn[ip] = pod_meta.nc_sn.values[i]
            break

In [ ]:
test_batch_ip = {}
for key in test_nc_sn:
    for i in range(ay49c.shape[0]):
        if ay49c.nc_sn.values[i] == test_nc_sn[key]:
            test_batch_ip[key] = ay49c.container_ip.values[i]
            break

In [ ]:
onoffline = pd.read_csv("/data/zkp/ump2/detailhost/onoffline.csv")
online = onoffline[onoffline.Task_Type=="Online"]
offline = onoffline[onoffline.Task_Type=="Offline"]

In [ ]:
# data = pd.merge(data, of, on=["sample_time", "nc_sn"], how="inner")
# ump2.drop(columns=["Unnamed: 0"], inplace=True)
DATA.head(1)

In [ ]:
# data = DATA[(DATA.qps__Q_s>=200) & (DATA.qps__Q_s<=230)]
# data = DATA[offline_data[:, 23]!=0]
data = DATA
data["rue"] = data.cpu_util__pct.values / data.qps__Q_s.values
data.shape

In [ ]:
# plt.plot(sorted(data.online_cpu_util__pct.values/96), np.arange(data.shape[0]) / (data.shape[0] - 1))
# plt.plot(sorted(data.offline_cpu_util__pct.values/96), np.arange(data.shape[0]) / (data.shape[0] - 1))
# plt.plot(sorted(data.machine_cpu_util__pct.values), np.arange(data.shape[0]) / (data.shape[0] - 1))
np.savetxt("/data/zkp/figure/ob2.txt", data[["machine_cpu_util__pct", "online_cpu_util__pct", "offline_cpu_util__pct"]].values)

In [ ]:
# _ = plt.hist(data.rue.values, bins=30, range=(0.02, 0.15))
# _ = plt.hist(data.machine_cpu_util__pct.values, bins=30, range=(60, 70))
_ = plt.hist(data.rt__ms_Q.values, bins=30, range=(0, 30))
plt.show()
# plt.scatter(data.rt__ms_Q.values, data.rue.values)
# for i in DATA.container_ip.unique()[:3]:
#     temp = DATA[DATA.container_ip==i]
#     plt.figure(figsize=(20, 8))
#     plt.plot(temp.sample_time, temp.rt__ms_Q)
#     cpu = temp.machine_cpu_util__pct.values
#     if np.mean(cpu)>20:
#         cpu /= 10
#     plt.plot(temp.sample_time, cpu)
#     plt.ylim(0, 20)
#     plt.show()
# temp = DATA[(DATA.sample_time>=1644076800+86400*1) & (DATA.sample_time<1644076800+86400*2)]
# rt = temp.rt__ms_Q.values
# print(np.percentile(rt, 80), np.percentile(rt, 20), np.mean(rt), np.std(rt))
# temp = DATA[(DATA.sample_time>=1644076800+86400*2) & (DATA.sample_time<1644076800+86400*3)]
# rt = temp.rt__ms_Q.values
# print(np.percentile(rt, 80), np.percentile(rt, 20), np.mean(rt), np.std(rt))
# plt.ylim(0, 20)
# plt.xlim(0, 20)
plt.plot(sorted(data.rt__ms_Q.values), np.arange(data.shape[0]) / (data.shape[0] - 1))
plt.plot([0, 1500], [0.50, 0.50])
plt.plot([0, 1500], [0.95, 0.95])
plt.plot([0, 1500], [0.05, 0.05])
plt.xlim(0, 20)

In [ ]:
# data = data[(data.machine_cpu_util__pct>=62) & (data.machine_cpu_util__pct<=64)]
# data = data[(data.machine_cpu_util__pct>=66) & (data.machine_cpu_util__pct<=70)]
data = DATA[(DATA.sample_time>=1644076800+86400*2) & (DATA.sample_time<1644076800+86400*3)]
# data = data[(data.qps__Q_s<150)]
data.shape

In [ ]:
y = data[metric].values
p = []
for i in range(5, 101, 5):
    p.append(np.percentile(y, i))

In [ ]:
y_new = []
for i in y:
    for j in range(1, 21):
        if i <= p[j-1]:
            y_new.append(j)
            break

In [ ]:
p

In [ ]:
model = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, y_new)
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
# predict = model.predict(X_test)
error = abs(predict - y_test)
plt.plot(sorted(error), np.arange(predict.shape[0]) / (predict.shape[0] - 1))
plt.xlim(0, 10)
plt.hlines(0.8, xmin=0, xmax=10)

In [ ]:
plt.figure(figsize=(20, 8))
plt.plot(range(0, predict.shape[0], 200), predict[::200])
plt.plot(range(0, predict.shape[0], 200), y_test[::200])

In [ ]:
metric = "rt__ms_Q"
p75 = np.percentile(data[metric].values, 95)
p25 = np.percentile(data[metric].values, 5)
# da = data[(data[metric] >= p75) | (data[metric] <= p25)]
da = data
da.shape, p75, p25

In [ ]:
X = []
for a in da.type.values:
    X.append(a)
X = np.asarray(X)
X.shape

In [ ]:
label = (da[metric].values >= p75)
label.shape
# label = []
# for i in da[metric].values:
#     if i <= p25 or i >= p75:
#         label.append(1)
# #     elif i >= p75:
# #         label.append(2)
#     else:
#         label.append(0)
# label = np.asarray(label)

In [ ]:
x0 = X[label==0]
y0 = label[label==0]
x1 = X[label==1]
y1 = label[label==1]
step = int((X.shape[0] / (X.shape[0]*0.1)) / 1.1)
x0 = x0[::step]
y0 = y0[::step]
X_ = np.vstack((x0, x1))
label_ = np.hstack((y0, y1))
X_.shape, label_.shape

In [ ]:
np.bincount(label_)

In [ ]:
model = RandomForestClassifier()
X_train, X_test, y_train, y_test = train_test_split(X_, label_)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
model.score(X_test, y_test)

In [ ]:
cnt = [[0, 0], [0, 0]]
for i in range(y_predict.shape[0]):
    cnt[int(y_test[i])][int(y_predict[i])] += 1
print(cnt)
score = [cnt[0][0] / sum(cnt[0]), cnt[1][1] / sum(cnt[1]), 
         cnt[0][0] / (cnt[0][0] + cnt[1][0]), cnt[1][1] / (cnt[1][1] + cnt[0][1])]
print(score)
print(2*score[0]*score[2] / (score[0] + score[2]), 2*score[1]*score[3] / (score[1]+score[3]))

In [ ]:
model.fit(X, label)
joblib.dump(model, "/data/zkp/ump2/detailhost/rt2.model")

In [ ]:
threhold = [8.308080628557107, 6.030347481642425]
label = []
for i in da.rt__ms_Q.values:
    if i <= threhold[1]:
        label.append(0)
    elif i >= threhold[0]:
        label.append(1)
    else:
        label.append(2)

In [ ]:
label.shape

In [ ]:
model = joblib.load("/data/zkp/ump2/detailhost/rt2.model")
y_predict = model.predict(X)

In [ ]:
# X = X[y_predict==1]
# rt = da.rt__ms_Q.values[y_predict==1]

In [ ]:
cnt = [[0, 0], [0, 0], [0, 0]]
for i in range(y_predict.shape[0]):
    cnt[int(label[i])][int(y_predict[i])] += 1
print(cnt)
score = [cnt[0][0] / sum(cnt[0]), cnt[1][1] / sum(cnt[1]), 
         cnt[0][0] / (cnt[0][0] + cnt[1][0]), cnt[1][1] / (cnt[1][1] + cnt[0][1])]
print(score)
print(2*score[0]*score[2] / (score[0] + score[2]), 2*score[1]*score[3] / (score[1]+score[3]))

In [ ]:
lr = LinearRegression()
lr.fit(X, data[metric].values)
lr.score(X, data[metric.values)
predict = lr.predict(X)
error = abs(predict - data[metric].values) / data[metric].values
plt.plot(sorted(error), np.arange(predict.shape[0]) / (predict.shape[0] - 1))

In [ ]:
# plt.hist(data[metric].values, bins=30)
plt.hist(data[data[metric]>=p75].qps__Q_s, bins=30)
plt.show()
plt.hist(data[data[metric]<=p25].qps__Q_s, bins=30)
plt.show()
plt.scatter(da[label==0].machine_cpu_util__pct, da[label==0].qps__Q_s)
plt.scatter(da[label==1].machine_cpu_util__pct, da[label==1].qps__Q_s)
plt.xlabel("Machine CPU")
plt.ylabel("QPS")
plt.legend(["0", "1"])

In [ ]:
heat0, heat1 = [], []
for i in range(y_test.shape[0]):
    if y_test[i] == y_predict[i]:
        if y_test[i] == 0:
            heat0.append(X_test[i])
        else:
            heat1.append(X_test[i])
heat0 = np.asarray(heat0)
heat1 = np.asarray(heat1)

In [ ]:
import seaborn as sns
vmin = 0
vmax = 60
cmap = "viridis_r"
fig, axes = plt.subplots(1, 2, figsize=(18, 8))
sns.heatmap(heat1.T, cmap=cmap, yticklabels=[], xticklabels=[], vmin=vmin, vmax=vmax, cbar=False, ax=axes[0])
# axes[0].xlabel("P80 Machine", fontsize=15)
# axes[0].ylabel("Task Type", fontsize=15)
# plt.show()
# plt.figure(figsize=(12, 8))
sns.heatmap(heat0.T, cmap=cmap, yticklabels=[], xticklabels=[], vmin=vmin, vmax=vmax, ax=axes[1])
# axes[1].xlabel("P20 Machine", fontsize=15)
# axes[1].ylabel("Task Type", fontsize=15)
plt.show()

In [ ]:
for i in range(heat0.shape[1]):
    print(i)
    plt.plot(sorted(heat0[:, i]), np.arange(heat0.shape[0]) / (heat0.shape[0] - 1))
    plt.plot(sorted(heat1[:, i]), np.arange(heat1.shape[0]) / (heat1.shape[0] - 1))
    plt.show()

In [ ]:
for i in range(60):
    x = []
    y = []
    temp = np.bincount(X[:, i])
    for j in range(temp.shape[0]):
        if temp[j] >= 30:
            cnt = np.bincount(label[X[:, i]==j])
            x.append(j)
            y.append(cnt[0] / temp[j])
    print(i)
    plt.plot(x, y)
    plt.xlabel("instance count")
    plt.ylabel("negative rate")
    plt.show()


In [ ]:
temp = X[:, 23] 
cnt = [0 for i in range(50)]
po = [0 for i in range(50)]
for i in range(temp.shape[0]):
    if temp[i] < 50:
        cnt[temp[i]] += 1
        if label[i]:
            po[temp[i]] += 1
for i in range(len(cnt)):
    print(cnt[i], po[i], end=', ')


In [ ]:
X59 = da[X[:, 27]==20]
Y59 = label[X[:, 27]==20]
plt.scatter(X59[Y59==0].machine_cpu_util__pct, X59[Y59==0].qps__Q_s)
plt.scatter(X59[Y59==1].machine_cpu_util__pct, X59[Y59==1].qps__Q_s)
plt.xlabel("Machine CPU")
plt.ylabel("QPS")
plt.legend(["0", "1"])

In [ ]:
batch = pd.DataFrame()
for i in range(7):
    temp = pd.read_csv("/data/zkp/ump2/detailhost/d" + str(i) + ".csv")
    temp.dropna(inplace=True)
    batch = pd.concat([batch, temp])

In [ ]:
batch.head(1)

In [ ]:
G = batch[["job_task_name", "fuxi_avg_cpu", "fuxi_avg_mem", "total_time"]].groupby(["job_task_name"]).mean()

In [ ]:
# G.head(1)
G.shape

In [ ]:
Merge = G[G.index.str.contains("Merge")]

In [ ]:
# len(Merge.index.unique()), Merge.shape
Merge.head(1)

In [ ]:
cpu = Merge.fuxi_avg_cpu.values
mem = Merge.fuxi_avg_mem.values
t = Merge.total_time.values

In [ ]:
y = plt.hist(cpu, bins=50, range=(0, 200))[0]
plt.show()
# plt.scatter(np.arange(y.shape[0])/10, np.log(y))
# plt.show()
y = plt.hist(mem, bins=50, range=(0, 2000))[0]
plt.show()
y = plt.hist(r, bins=50, range=(0, 200))[0]
plt.show()

In [ ]:
task_type = {}
type_cnt = [0 for i in range(60)]
cpu = G.fuxi_avg_cpu.values
mem = G.fuxi_avg_mem.values
t = G.total_time.values
key = G.index.values
for i in range(G.shape[0]):
    index = 0
    if cpu[i] < 50:
        idnex = 0
    elif 50 <= cpu[i] < 150:
        index = 1
    else:
        index = 2

    if mem[i] < 512:
        index = (index*5) + 0
    elif 512 <= mem[i] < 1536:
        index = (index*5) + 1
    elif 1536 <= mem[i] < 3072:
        index = (index*5) + 2
    elif 3072 <= mem[i] < 6144:
        index = (index*5) + 3
    else:
        index = (index*5) + 4

    if t[i] < 20:
        index = (index*4) + 0
    elif 20 <= t[i] < 40:
        index = (index*4) + 1
    elif 40 <= t[i] < 100:
        index = (index*4) + 2
    else:
        index = (index*4) + 3
    task_type[key[i]] = index
    type_cnt[index] += 1

In [ ]:
temp = []
for i in batch.job_task_name.values:
    temp.append(task_type[i])
batch["type"] = temp

In [ ]:
batch

In [ ]:
DATA

In [ ]:
offline_data = []
sample_time = DATA.sample_time.values
container_ip = DATA.container_ip.values
cur_ip = container_ip[0]
df = batch[batch.worker_ip==test_batch_ip[cur_ip]]
for i in range(DATA.shape[0]):
#     print(i, end=',')
#     start = time.time()
    if cur_ip != container_ip[i]:
        print(cur_ip, end=',')
        cur_ip = container_ip[i]
        df = batch[batch.worker_ip==test_batch_ip[cur_ip]]  
    maxpd = df[df.start_time<sample_time[i]]
    maxpd = maxpd[maxpd.end_time>sample_time[i]-60]
#     print(time.time()-start)
    offline_data.append(np.bincount(maxpd.type.values, minlength=60))


In [ ]:
DATA["type"] = offline_data

In [ ]:
DATA.to_csv("/data/zkp/ump2/detailhost/DATA.csv", index=False)

In [ ]:
offline_data = []
for tmax in data.index.values:
    maxpd = batch[~((batch.start_time>=tmax) | (batch.end_time<=tmax-60))]
    total_time = []
    start = maxpd.start_time.values
    end = maxpd.end_time.values
    for i in range(maxpd.shape[0]):
        if end[i] >= tmax and start[i] <= tmax-60:
            total_time.append(60)
        elif end[i] >= tmax and start[i] > tmax-60:
            total_time.append(tmax-start[i])
        elif end[i] < tmax and start[i] <= tmax-60:
            total_time.append(end[i]-tmax+60)
        else:
            total_time.append(end[i]-start[i])
    maxpd["total_time"] = total_time
    maxpd["cpu_weight"] = maxpd["total_time"] * maxpd["fuxi_avg_cpu"] / np.sum(maxpd["total_time"] * maxpd["fuxi_avg_cpu"]) * 100
    maxpd["mem_weight"] = maxpd["total_time"] * maxpd["fuxi_avg_mem"] / np.sum(maxpd["total_time"] * maxpd["fuxi_avg_mem"]) * 100
    maxpd["rate"] = maxpd["mem_weight"] / maxpd["cpu_weight"]
    if not maxpd.shape[0]:
        offline_data.append([0 for i in range(9)])
        continue
    offline_temp = []
    offline_temp.append(maxpd.shape[0]) # instance_num
    offline_temp.append(len(maxpd.fuxi_job_name.unique())) # job_num
    offline_temp.append(len((maxpd.fuxi_job_name+maxpd.fuxi_task_name).unique())) # task_num
    offline_temp.append(np.sum(maxpd.total_time * maxpd.fuxi_avg_cpu) / (9600 * 60)) # offline_cpu_pct
    offline_temp.append(np.sum(maxpd.total_time * maxpd.fuxi_avg_mem) / (512*1024*60)) # offline_mem_pct
    offline_temp.append(np.sum(maxpd.fuxi_avg_mem/maxpd.fuxi_avg_cpu*maxpd.mem_weight)) # score1
    offline_temp.append(np.sum(maxpd.rate*maxpd.mem_weight)) # score2
    offline_temp.append(np.sum(maxpd[maxpd.rate>=2].rate * maxpd[maxpd.rate>=2].mem_weight)) # mem_score
    offline_temp.append(np.sum(maxpd[maxpd.rate<=0.5].rate * maxpd[maxpd.rate<=0.5].cpu_weight)) # cpu_score
    offline_data.append(offline_temp)
offline_data = np.asarray(offline_data)

In [ ]:
# columns = ["total_cpu", "total_mem", "total_cpu_rank", "total_mem_rank"]
# for i in range(len(columns)):
#     data[columns[i]] = offline_data[:, i]

In [ ]:
rate = [[], [], [], []]
for tmax in data.index.values:
    maxpd = batch[~((batch.start_time>=tmax) | (batch.end_time<=tmax-60))]
    if not maxpd.shape[0]:
        continue
    total_time = []
    start = maxpd.start_time.values
    end = maxpd.end_time.values
    for i in range(maxpd.shape[0]):
        if end[i] >= tmax and start[i] <= tmax-60:
            total_time.append(60)
        elif end[i] >= tmax and start[i] > tmax-60:
            total_time.append(tmax-start[i])
        elif end[i] < tmax and start[i] <= tmax-60:
            total_time.append(end[i]-tmax+60)
        else:
            total_time.append(end[i]-start[i])
    maxpd["total_time"] = total_time
    maxpd["cpu_weight"] = maxpd["total_time"] * maxpd["fuxi_avg_cpu"] / np.sum(maxpd["total_time"] * maxpd["fuxi_avg_cpu"]) * 100
    maxpd["mem_weight"] = maxpd["total_time"] * maxpd["fuxi_avg_mem"] / np.sum(maxpd["total_time"] * maxpd["fuxi_avg_mem"]) * 100
    maxpd["rate"] = maxpd["mem_weight"] / maxpd["cpu_weight"]
    task_name = maxpd.fuxi_task_name.values
    r = maxpd.rate.values
    for i in range(maxpd.shape[0]):
        if task_name[i][0] == 'M' and task_name[i][1].isdigit():
            rate[0].append(r[i])
        elif task_name[i][0] == 'R' and task_name[i][1].isdigit():
            rate[1].append(r[i])
        elif task_name[i][0] == 'J' and task_name[i][1].isdigit():
            rate[2].append(r[i])
        else:
            rate[3].append(r[i])


In [ ]:
for i in range(len(rate)):
    if np.isnan(rate[i]):
        rate[i] = 0
    elif np.isinf(rate[i]):
        rate[i] = 0

In [ ]:
y = plt.hist(rate, bins=50, range=(0, 5))[0]

In [ ]:
plt.scatter(np.arange(y.shape[0])/10, np.log(y))

In [ ]:
# y = [[], [], [], []]
# for r in range(len(rate)):
y = plt.hist(rate[0], bins=50, range=(0, 5))[0]
plt.show()
plt.scatter(np.arange(y.shape[0])/10, np.log(y))
plt.show()
y = plt.hist(rate[1], bins=50, range=(0, 5))[0]
plt.show()
plt.scatter(np.arange(y.shape[0])/10, np.log(y))
plt.show()
y = plt.hist(rate[2], bins=50, range=(0, 5))[0]
plt.show()
plt.scatter(np.arange(y.shape[0])/10, np.log(y))
plt.show()
y = plt.hist(rate[3], bins=50, range=(0, 5))[0]
plt.show()
plt.scatter(np.arange(y.shape[0])/10, np.log(y))
plt.show()

In [ ]:
mu = np.mean(rate)
sigma = np.std(rate)
print(mu, sigma)
mu + sigma, mu - sigma

In [ ]:
columns = ["instance_num", "job_num", "task_num", "offline_cpu_pct", "offline_mem_pct", "score1", "score2", "mem_score", "cpu_score"]
for i in range(len(columns)):
    data[columns[i]] = offline_data[:, i]

In [ ]:
data.tail(10)

In [ ]:
tmax=1644385800
tmin=1644386220

In [ ]:
minpd = batch[~((batch.start_time>tmin) | (batch.end_time<tmin-60))]
total_time = []
start = minpd.start_time.values
end = minpd.end_time.values
for i in range(minpd.shape[0]):
    if end[i] >= tmin and start[i] <= tmin-60:
        total_time.append(60)
    elif end[i] >= tmin and start[i] > tmin-60:
        total_time.append(tmin-start[i])
    elif end[i] < tmin and start[i] <= tmin-60:
        total_time.append(end[i]-tmin+60)
    else:
        total_time.append(end[i]-start[i])
minpd["total_time"] = total_time
minpd["cpu_weight"] = minpd["total_time"] * minpd["fuxi_avg_cpu"] / np.sum(minpd["total_time"] * minpd["fuxi_avg_cpu"]) * 100
minpd["mem_weight"] = minpd["total_time"] * minpd["fuxi_avg_mem"] / np.sum(minpd["total_time"] * minpd["fuxi_avg_mem"]) * 100
minpd["rate"] = minpd["mem_weight"] / minpd["cpu_weight"]

In [ ]:
print(minpd.shape[0], len(minpd.fuxi_job_name.unique()), len((minpd.fuxi_job_name+minpd.fuxi_task_name).unique()))
print(np.sum(minpd["total_time"] * minpd["fuxi_avg_mem"]) / (512*1024*60), np.sum(minpd.total_time * minpd.fuxi_avg_cpu) / (9600*60))
print(np.sum(minpd.fuxi_avg_mem/minpd.fuxi_avg_cpu*minpd.mem_weight), np.sum(minpd.rate*minpd.mem_weight))
print("mem socre: ", np.sum(minpd[minpd.rate>=2].rate * minpd[minpd.rate>=2].mem_weight))
print("cpu socre: ", np.sum(minpd[minpd.rate<=0.5].rate * minpd[minpd.rate<=0.5].cpu_weight))
minpd.sort_values(by=["start_time"], ascending=False).head(50)

In [ ]:
maxpd = batch[~((batch.start_time>tmax) | (batch.end_time<tmax-60))]
total_time = []
start = maxpd.start_time.values
end = maxpd.end_time.values
for i in range(maxpd.shape[0]):
    if end[i] >= tmax and start[i] <= tmax-60:
        total_time.append(60)
    elif end[i] >= tmax and start[i] > tmax-60:
        total_time.append(tmax-start[i])
    elif end[i] < tmax and start[i] <= tmax-60:
        total_time.append(end[i]-tmax+60)
    else:
        total_time.append(end[i]-start[i])
maxpd["total_time"] = total_time
maxpd["cpu_weight"] = maxpd["total_time"] * maxpd["fuxi_avg_cpu"] / np.sum(maxpd["total_time"] * maxpd["fuxi_avg_cpu"]) * 100
maxpd["mem_weight"] = maxpd["total_time"] * maxpd["fuxi_avg_mem"] / np.sum(maxpd["total_time"] * maxpd["fuxi_avg_mem"]) * 100
maxpd["rate"] = maxpd["mem_weight"] / maxpd["cpu_weight"]

In [ ]:
print(maxpd.shape[0], len(maxpd.fuxi_job_name.unique()), len((maxpd.fuxi_job_name+maxpd.fuxi_task_name).unique()))
print(np.sum(maxpd["total_time"] * maxpd["fuxi_avg_mem"]) / (512*1024*60), np.sum(maxpd.total_time * maxpd.fuxi_avg_cpu) / (9600 * 60))
print(np.sum(maxpd.fuxi_avg_mem/maxpd.fuxi_avg_cpu*maxpd.mem_weight), np.sum(maxpd.rate*maxpd.mem_weight))
print("mem socre: ", np.sum(maxpd[maxpd.rate>=2].rate * maxpd[maxpd.rate>=2].mem_weight))
print("cpu socre: ", np.sum(maxpd[maxpd.rate<=0.5].rate * maxpd[maxpd.rate<=0.5].cpu_weight))
maxpd.sort_values(by=["mem_weight"], ascending=False).head(50)

In [ ]:
plt.figure(figsize=(10, 8))
X, y = [[], [], [], []], [[], [], [], []]
task_name = minpd.fuxi_task_name.values
cpu_weight = minpd.cpu_weight.values
mem_weight = minpd.mem_weight.values
for i in range(minpd.shape[0]):
    if task_name[i][0] == 'M' and task_name[i][1].isdigit():
        X[0].append(cpu_weight[i])
        y[0].append(mem_weight[i])
    elif task_name[i][0] == 'R' and task_name[i][1].isdigit():
        X[1].append(cpu_weight[i])
        y[1].append(mem_weight[i])
    elif task_name[i][0] == 'J' and task_name[i][1].isdigit():
        X[2].append(cpu_weight[i])
        y[2].append(mem_weight[i])
    else:
        X[3].append(cpu_weight[i])
        y[3].append(mem_weight[i])
for i, m in zip(range(4), ['*', '+', 'x', 'o']):
    plt.scatter(X[i], y[i], marker=m, c="blue")

X, y = [[], [], [], []], [[], [], [], []]
task_name = maxpd.fuxi_task_name.values
cpu_weight = maxpd.cpu_weight.values
mem_weight = maxpd.mem_weight.values
for i in range(maxpd.shape[0]):
    if task_name[i][0] == 'M' and task_name[i][1].isdigit():
        X[0].append(cpu_weight[i])
        y[0].append(mem_weight[i])
    elif task_name[i][0] == 'R' and task_name[i][1].isdigit():
        X[1].append(cpu_weight[i])
        y[1].append(mem_weight[i])
    elif task_name[i][0] == 'J' and task_name[i][1].isdigit():
        X[2].append(cpu_weight[i])
        y[2].append(mem_weight[i])
    else:
        X[3].append(cpu_weight[i])
        y[3].append(mem_weight[i])
for i, m in zip(range(4), ['*', '+', 'x', 'o']):
    plt.scatter(X[i], y[i], marker=m, c="red")
    
plt.xlabel("CPU%", fontsize=12)
plt.ylabel("MEM%", fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(["M", "R", "J", "Min", "M", "R", "J", "Max"], ncol=8, fontsize=12, loc="upper left")